In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from time import time
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score 
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA

In [2]:
df = pd.read_csv('final_df2.csv',index_col=0)

In [3]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [4]:
from sklearn.preprocessing import StandardScaler

std_df = StandardScaler().fit_transform(X)
std_df = pd.DataFrame(std_df, index=X.index, columns=X.columns)
df2 = std_df
df2.head()

,ELMO2,CREB3L1,RPS11,PNMA1,MMP2,C10orf90,ZHX3,ERCC5,GPR98,RXFP3,...,SLC2A11,GRIP2,GPLD1,RAB8A,RXFP2,PIK3IP1,SLC39A6,SNRPD2,AQP7,CTSC
0,0.886827,0.349044,0.159261,-1.586876,-0.783517,-0.651792,-0.192552,-0.688943,1.790315,0.606942,...,0.618291,-0.244858,0.087421,-0.441970,0.805609,0.084007,-0.853409,-0.767943,-0.097567,0.208945
1,-0.076357,0.902533,0.026440,1.624105,1.396133,0.195322,0.394281,0.733565,1.460883,-0.288978,...,0.083736,-0.957388,0.095699,0.972672,0.691112,1.004964,-0.178898,-0.945141,0.843130,-1.062326
2,1.160544,0.875732,4.027129,0.072768,0.488563,-0.707819,-1.258655,2.353938,1.514641,-0.194159,...,0.561794,-0.680456,-0.348212,0.766775,-0.167253,0.878351,1.027238,-0.487267,0.630930,-0.273803
3,-0.093755,0.023215,-1.310286,0.819120,-0.815949,-0.407544,0.793497,-0.158602,-0.161729,1.060372,...,2.103530,-1.246807,-1.676863,0.213831,-1.585708,0.933448,0.740015,-1.050423,-0.987469,-0.644158
4,-0.249586,-0.633865,-0.325074,0.155425,0.019339,-0.462644,-0.576603,-1.569976,-1.943742,0.022223,...,-1.478282,0.569776,-0.794047,-0.079013,0.559841,-0.793164,-1.567624,0.733505,-0.957059,0.428773


In [5]:
from sklearn.decomposition import PCA

pca = PCA(n_components=533)
pca_array = pca.fit_transform(X)


In [6]:
pca = PCA()
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)
# 주성분의 개수를 지정하지 않고 찾아줌
# 분산의 비율을 누적해서 더한 값이 95% 이상인 주성분의 개수
# 즉, 분산의 95%이상 유지
d = np.argmax(cumsum >= 0.99) + 1
d

533

In [ ]:
dec=PCA(0.99)
x2=dec.fit_transform(X)
x2.shape

In [9]:
from sklearn.model_selection import train_test_split
x, x_test, y, y_test=train_test_split(pca_array,y,test_size= 0.2, random_state=42)

In [10]:
def estimators_repeater(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC()],tr_slicer=(None,None),tst_slicer=(None,None),loops=300,scorer=recall_score,x=x,y=y):
    training_score={}
    testing_score={}
    timing={}
    for cl in estimators:
        cl_name=cl.__class__.__name__
        training_score[cl_name]=[]
        testing_score[cl_name]=[]
        timing[cl_name]=[]
    for i in range(loops):
        k1=time()
        x_train, x_test, y_train, y_test=train_test_split(x,y,random_state=i)
        for cl in estimators:
            a=time()
            cl_name=cl.__class__.__name__
            clean_cl=clone(cl)
            clean_cl.fit(x_train[tr_slicer[0]:tr_slicer[1]],y_train[tr_slicer[0]:tr_slicer[1]])
            training_score[cl_name].append(scorer(y_train[tr_slicer[0]:tr_slicer[1]],clean_cl.predict(x_train[tr_slicer[0]:tr_slicer[1]])))
            testing_score[cl_name].append(scorer(y_test[tst_slicer[0]:tst_slicer[1]],clean_cl.predict(x_test[tst_slicer[0]:tst_slicer[1]])))
            b=time()                                
            timing[cl_name].append(b-a)
        k2=time()
        print(f'number {i} out of {loops} took {k2-k1} seconds')
    global training_score_df
    training_score_df=pd.DataFrame(training_score)
    global testing_score_df
    testing_score_df=pd.DataFrame(testing_score)
    global timing_df
    timing_df=pd.DataFrame(timing)

In [11]:
estimators_repeater(estimators=[RandomForestClassifier(),AdaBoostClassifier(),SVC(),LogisticRegression()],x=x,y=y)

number 0 out of 300 took 1.475097417831421 seconds
number 1 out of 300 took 1.4551093578338623 seconds
number 2 out of 300 took 1.4072363376617432 seconds
number 3 out of 300 took 1.4491252899169922 seconds
number 4 out of 300 took 1.3992581367492676 seconds
number 5 out of 300 took 1.5329005718231201 seconds
number 6 out of 300 took 1.4331681728363037 seconds
number 7 out of 300 took 1.4062395095825195 seconds
number 8 out of 300 took 1.4890177249908447 seconds
number 9 out of 300 took 1.38529634475708 seconds
number 10 out of 300 took 1.4481275081634521 seconds
number 11 out of 300 took 1.408233880996704 seconds
number 12 out of 300 took 1.4202024936676025 seconds
number 13 out of 300 took 1.3872904777526855 seconds
number 14 out of 300 took 1.3942711353302002 seconds
number 15 out of 300 took 1.474057912826538 seconds
number 16 out of 300 took 1.3733665943145752 seconds
number 17 out of 300 took 1.8172016143798828 seconds
number 18 out of 300 took 1.344404935836792 seconds
number 19

C:\Users\R301-9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


number 27 out of 300 took 1.5019831657409668 seconds
number 28 out of 300 took 1.773259162902832 seconds
number 29 out of 300 took 1.479043960571289 seconds
number 30 out of 300 took 1.4122238159179688 seconds
number 31 out of 300 took 1.4650821685791016 seconds
number 32 out of 300 took 1.5747885704040527 seconds
number 33 out of 300 took 1.6236584186553955 seconds
number 34 out of 300 took 1.4182078838348389 seconds
number 35 out of 300 took 1.8580310344696045 seconds
number 36 out of 300 took 1.5678093433380127 seconds
number 37 out of 300 took 1.5239245891571045 seconds
number 38 out of 300 took 1.6625549793243408 seconds
number 39 out of 300 took 1.5019824504852295 seconds
number 40 out of 300 took 1.579775333404541 seconds
number 41 out of 300 took 1.417210340499878 seconds
number 42 out of 300 took 1.4650824069976807 seconds
number 43 out of 300 took 1.3643510341644287 seconds
number 44 out of 300 took 1.5019845962524414 seconds
number 45 out of 300 took 1.422196865081787 second

C:\Users\R301-9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


number 114 out of 300 took 1.4022502899169922 seconds
number 115 out of 300 took 1.8161439895629883 seconds
number 116 out of 300 took 1.4521172046661377 seconds
number 117 out of 300 took 1.456106185913086 seconds
number 118 out of 300 took 1.410229206085205 seconds
number 119 out of 300 took 1.4350664615631104 seconds
number 120 out of 300 took 1.4848461151123047 seconds
number 121 out of 300 took 1.5174028873443604 seconds
number 122 out of 300 took 1.600719690322876 seconds
number 123 out of 300 took 1.632634162902832 seconds
number 124 out of 300 took 1.5239253044128418 seconds
number 125 out of 300 took 1.3852951526641846 seconds
number 126 out of 300 took 1.4401490688323975 seconds
number 127 out of 300 took 1.4231946468353271 seconds
number 128 out of 300 took 1.52691650390625 seconds
number 129 out of 300 took 1.52292799949646 seconds
number 130 out of 300 took 1.3932740688323975 seconds
number 131 out of 300 took 1.4600956439971924 seconds
number 132 out of 300 took 1.3892853

...

number 205 out of 300 took 1.3952693939208984 seconds
number 206 out of 300 took 1.4461324214935303 seconds
number 207 out of 300 took 1.370335340499878 seconds
number 208 out of 300 took 1.450122594833374 seconds
number 209 out of 300 took 1.4062397480010986 seconds
number 210 out of 300 took 1.389286756515503 seconds
number 211 out of 300 took 1.4132189750671387 seconds
number 212 out of 300 took 1.3952693939208984 seconds
number 213 out of 300 took 1.438154935836792 seconds
number 214 out of 300 took 1.3962666988372803 seconds
number 215 out of 300 took 1.4261856079101562 seconds
number 216 out of 300 took 1.4082343578338623 seconds
number 217 out of 300 took 1.611689805984497 seconds
number 218 out of 300 took 1.5229277610778809 seconds
number 219 out of 300 took 1.4810395240783691 seconds
number 220 out of 300 took 1.469071865081787 seconds
number 221 out of 300 took 1.3942716121673584 seconds
number 222 out of 300 took 1.443140983581543 seconds
number 223 out of 300 took 1.384299

C:\Users\R301-9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


number 235 out of 300 took 1.3922767639160156 seconds
number 236 out of 300 took 1.4192049503326416 seconds
number 237 out of 300 took 1.43117356300354 seconds
number 238 out of 300 took 1.3793115615844727 seconds
number 239 out of 300 took 1.4581010341644287 seconds
number 240 out of 300 took 1.3743255138397217 seconds
number 241 out of 300 took 1.4052419662475586 seconds
number 242 out of 300 took 1.4162132740020752 seconds
number 243 out of 300 took 1.3763198852539062 seconds
number 244 out of 300 took 1.4221971035003662 seconds
number 245 out of 300 took 1.3793110847473145 seconds
number 246 out of 300 took 1.562821626663208 seconds
number 247 out of 300 took 1.3962655067443848 seconds
number 248 out of 300 took 1.3543784618377686 seconds
number 249 out of 300 took 1.452117919921875 seconds
number 250 out of 300 took 1.4102280139923096 seconds
number 251 out of 300 took 1.4341654777526855 seconds
number 252 out of 300 took 1.411226749420166 seconds
number 253 out of 300 took 1.3872

C:\Users\R301-9\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


number 284 out of 300 took 1.3872902393341064 seconds
number 285 out of 300 took 1.441145896911621 seconds
number 286 out of 300 took 1.377316951751709 seconds
number 287 out of 300 took 1.4341650009155273 seconds
number 288 out of 300 took 1.3982610702514648 seconds
number 289 out of 300 took 1.3793118000030518 seconds
number 290 out of 300 took 1.4012529850006104 seconds
number 291 out of 300 took 1.3713328838348389 seconds
number 292 out of 300 took 1.4401497840881348 seconds
number 293 out of 300 took 1.3753223419189453 seconds
number 294 out of 300 took 1.4122238159179688 seconds
number 295 out of 300 took 1.4202020168304443 seconds
number 296 out of 300 took 1.3663463592529297 seconds
number 297 out of 300 took 1.4241926670074463 seconds
number 298 out of 300 took 1.3683404922485352 seconds
number 299 out of 300 took 1.3942713737487793 seconds


In [12]:
training_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.0,300.0,300.0,300.0
mean,1.0,1.0,1.0,1.0
std,0.0,0.0,0.0,0.0
min,1.0,1.0,1.0,1.0
25%,1.0,1.0,1.0,1.0
50%,1.0,1.0,1.0,1.0
75%,1.0,1.0,1.0,1.0
max,1.0,1.0,1.0,1.0


In [13]:
testing_score_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.0,300.000000,300.000000,300.000000
mean,1.0,0.995373,0.997644,0.997644
std,0.0,0.006994,0.004053,0.004053
min,1.0,0.954128,0.989796,0.989796
25%,1.0,0.990741,0.991228,0.991228
50%,1.0,1.000000,1.000000,1.000000
75%,1.0,1.000000,1.000000,1.000000
max,1.0,1.000000,1.000000,1.000000


In [14]:
timing_df.describe()

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
count,300.000000,300.000000,300.000000,300.000000
mean,0.396108,0.989312,0.030262,0.034494
std,0.025920,0.065970,0.005698,0.015542
min,0.347072,0.932507,0.025903,0.026899
25%,0.382000,0.941506,0.027927,0.029919
50%,0.389985,0.966407,0.028950,0.031886
75%,0.404924,1.009530,0.030918,0.033909
max,0.595408,1.370314,0.113695,0.279254


In [15]:
testing_score_df

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
0,1.0,1.000000,1.000000,1.000000
1,1.0,1.000000,1.000000,1.000000
2,1.0,0.990826,1.000000,1.000000
3,1.0,1.000000,1.000000,1.000000
4,1.0,1.000000,1.000000,1.000000
...,...,...,...,...
295,1.0,1.000000,1.000000,1.000000
296,1.0,0.981308,0.990654,0.990654
297,1.0,1.000000,1.000000,1.000000
298,1.0,1.000000,1.000000,1.000000


In [16]:
testing_score_df.sort_values(by='SVC',ascending=False)

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
0,1.0,1.000000,1.000000,1.000000
128,1.0,1.000000,1.000000,1.000000
170,1.0,1.000000,1.000000,1.000000
171,1.0,1.000000,1.000000,1.000000
173,1.0,0.990654,1.000000,1.000000
...,...,...,...,...
82,1.0,0.990291,0.990291,0.990291
138,1.0,0.990196,0.990196,0.990196
183,1.0,1.000000,0.990196,0.990196
41,1.0,1.000000,0.990099,0.990099


In [17]:
testing_score_df.sort_values(by='RandomForestClassifier',ascending=False)

,RandomForestClassifier,AdaBoostClassifier,SVC,LogisticRegression
0,1.0,1.000000,1.0,1.0
206,1.0,1.000000,1.0,1.0
204,1.0,1.000000,1.0,1.0
203,1.0,1.000000,1.0,1.0
202,1.0,1.000000,1.0,1.0
...,...,...,...,...
99,1.0,1.000000,1.0,1.0
98,1.0,0.990654,1.0,1.0
97,1.0,1.000000,1.0,1.0
96,1.0,1.000000,1.0,1.0


In [18]:
prediction=SVC(random_state=150)

In [19]:
prediction.fit(x,y)

SVC(random_state=150)

In [20]:
prediction.score(x_test,y_test)

0.9915254237288136

In [65]:
pca.explained_variance_ratio_
sum(pca.explained_variance_ratio_)

0.993685152999509

In [33]:
df2.shape[1]

17815

In [46]:
from sklearn.decomposition import PCA

pca = PCA(n_components=590)
pca_array = pca.fit_transform(df2)
pca_df = pd.DataFrame(pca_array, index=df2.index,
                      columns=[f"pca{num+1}" for num in range(590)])
pca_df.head()

,pca1,pca2,pca3,pca4,pca5,pca6,pca7,pca8,pca9,pca10,...,pca581,pca582,pca583,pca584,pca585,pca586,pca587,pca588,pca589,pca590
0,-33.804476,-2.787982,-5.770776,-30.958220,5.887185,13.407046,2.691613,-14.574579,-9.552975,35.365711,...,0.196188,0.154316,0.327665,0.057420,0.454254,0.043743,-0.494110,0.163552,0.313049,2.829963e-15
1,12.129357,-20.493103,13.751279,-29.977843,-8.738578,-3.651478,-16.750653,18.264358,0.222539,-23.017786,...,-1.231051,0.436648,0.278043,0.841787,0.056323,0.110008,0.127392,0.172217,-0.086814,2.829963e-15
2,-4.647118,-27.232655,-10.584373,-4.731128,-26.303871,-22.174773,5.284348,1.192801,23.295138,2.967032,...,-0.016966,0.286153,0.738451,-0.027324,0.026627,-0.631649,0.330128,-0.128202,0.091551,2.829963e-15
3,-7.638704,-57.561760,7.498658,5.308718,0.283737,-2.635175,-8.665971,1.103797,-17.317805,-5.960301,...,0.261537,-0.382600,0.365680,0.041219,0.090431,-0.025359,0.661129,0.442092,-0.349987,2.829963e-15
4,-31.522639,76.837420,-26.187440,-6.820377,12.824112,-1.214566,11.631132,-9.568467,-10.434659,-4.633602,...,-0.150812,0.078911,0.046604,-0.195311,0.079283,-0.053218,0.263967,0.386297,-0.499102,2.829963e-15


In [49]:
result = pd.DataFrame({'설명가능한 분산 비율(고윳값)':pca.explained_variance_,
             '기여율':pca.explained_variance_ratio_},
            index=np.array([f"pca{num+1}" for num in range(590)]))
result['누적기여율'] = result['기여율'].cumsum()
result

,설명가능한 분산 비율(고윳값),기여율,누적기여율
pca1,1.553460e+03,8.705178e-02,0.087052
pca2,1.347472e+03,7.550873e-02,0.162561
pca3,7.642334e+02,4.282560e-02,0.205386
pca4,6.858864e+02,3.843524e-02,0.243821
pca5,4.964501e+02,2.781974e-02,0.271641
...,...,...,...
pca586,2.475582e+00,1.387250e-04,0.999615
pca587,2.370171e+00,1.328181e-04,0.999748
pca588,2.318629e+00,1.299298e-04,0.999878
pca589,2.185088e+00,1.224465e-04,1.000000


In [51]:
# MNIST 데이터 사용하기
from sklearn.datasets import fetch_openml

# 784개의 픽셀(특성)을 가지고 있는 데이터 셋
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.target = mnist.target.astype(np.uint8)

KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split

X = mnist["data"]
y = mnist["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y)